In [34]:
import pandas as pd 
import numpy as np
import numpy.linalg as nlg
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import StandardScaler
from factor_analyzer import FactorAnalyzer, calculate_kmo, calculate_bartlett_sphericity  
from scipy.stats import bartlett


In [2]:
data = pd.read_csv('covid_data.csv')

In [3]:
data.head()

,submission_id,observation_id,observed_time,user_id,form_id,form_version,form_name,form_tier,campaign_id,campaign_name,...,how_have_you_been_getting_updates_on_covid_19_from_the_list_below_select_all_the_types_of_media_youu2019ve_been_relying_on_for_updates_as_coronavirus_cases_increase,how_much_do_you_agree_with_the_following_statement_u201ci_am_well_informed_about_covid_19_and_understand_where_to_get_additional_informationu201d,how_satisfied_are_you_with_the_media_coverage_of_covid_19,in_the_past_6_months_have_you_heard_read_or_seen_any_information_or_messaging_that_highlights_the_downsides_of_a_covid_19_vaccine,which_media_outlets_do_you_think_only_report_on_the_potential_disadvantages_of_a_covid_19_vaccine,how_safe_do_you_believe_the_covid_19_vaccines_being_developed_now_are,which_of_the_following_countries_do_you_feel_the_most_ready_to_receive_a_covid_19_vaccine_from,how_much_do_you_agree_with_the_following_statement_u201cvaccines_are_necessary_to_promote_herd_immunity_and_ensure_the_health_and_safety_of_everyone_in_the_community_including_immunocompromised_individuals_u201d,when_a_covid_19_vaccine_becomes_widely_publicly_available_how_likely_are_you_to_get_it,how_quickly_will_you_get_the_covid_19_vaccine
0,6639385295716352,6639385295716352,2020-07-29 11:31:42.584 UTC,4511945684418560,4850691271294976,102,COVID-19 Impact,1,6683622704939008,bj_covid-19-impact,...,"Health Organizations (e.g. WHO, CDC, local hea...",Neither agree nor disagree,Neither satisfied nor dissatisfied,did not exist,did not exist,did not exist,did not exist,did not exist,did not exist,did not exist
1,6400325797019648,6400325797019648,2020-10-02 17:47:36.835 UTC,5821834429988864,4850691271294976,102,COVID-19 Impact,1,6683622704939008,bj_covid-19-impact,...,Local newspaper (or its website)^Local televis...,Agree,Satisfied,did not exist,did not exist,did not exist,did not exist,did not exist,did not exist,did not exist
2,4534453236137984,4534453236137984,2020-05-05 20:59:35.722 UTC,4695912256897024,4850691271294976,102,COVID-19 Impact,1,6683622704939008,bj_covid-19-impact,...,Local newspaper (or its website)^Local radio^L...,Agree,Satisfied,did not exist,did not exist,did not exist,did not exist,did not exist,did not exist,did not exist
3,5919602254282752,5919602254282752,2020-09-23 21:57:33.928 UTC,5140769135919104,4850691271294976,102,COVID-19 Impact,1,6683622704939008,bj_covid-19-impact,...,Local newspaper (or its website),Agree,Satisfied,did not exist,did not exist,did not exist,did not exist,did not exist,did not exist,did not exist
4,6163379149602816,6163379149602816,2020-12-02 07:34:41.194 UTC,5108927901728768,4850691271294976,102,COVID-19 Impact,1,6683622704939008,bj_covid-19-impact,...,Local newspaper (or its website)^Local televis...,Agree,Neither satisfied nor dissatisfied,did not exist,did not exist,did not exist,did not exist,did not exist,did not exist,did not exist


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2866985 entries, 0 to 2866984
Data columns (total 89 columns):
 #   Column                                                                                                                                                                                                               Dtype  
---  ------                                                                                                                                                                                                               -----  
 0   submission_id                                                                                                                                                                                                        int64  
 1   observation_id                                                                                                                                                                                              

In [5]:
len(data)

2866985

In [6]:
#find the three questions
df = data[['how_much_of_an_impact_do_you_believe_the_covid_19_pandemic_and_subsequent_quarantine_measures_have_had_on_your_mental_and_emotional_health',
          'how_much_do_you_agree_with_the_following_statement_u201cvaccines_are_necessary_to_promote_herd_immunity_and_ensure_the_health_and_safety_of_everyone_in_the_community_including_immunocompromised_individuals_u201d',
          'how_much_do_you_agree_with_the_following_statement_u201ci_am_well_informed_about_covid_19_and_understand_where_to_get_additional_informationu201d']]
df

,how_much_of_an_impact_do_you_believe_the_covid_19_pandemic_and_subsequent_quarantine_measures_have_had_on_your_mental_and_emotional_health,how_much_do_you_agree_with_the_following_statement_u201cvaccines_are_necessary_to_promote_herd_immunity_and_ensure_the_health_and_safety_of_everyone_in_the_community_including_immunocompromised_individuals_u201d,how_much_do_you_agree_with_the_following_statement_u201ci_am_well_informed_about_covid_19_and_understand_where_to_get_additional_informationu201d
0,Neither positive nor negative impact,did not exist,Neither agree nor disagree
1,Extreme negative impact,did not exist,Agree
2,Some negative impact,did not exist,Agree
3,Some negative impact,did not exist,Agree
4,Some negative impact,did not exist,Agree
...,...,...,...
2866980,Neither positive nor negative impact,did not exist,Neither agree nor disagree
2866981,Some negative impact,did not exist,Agree
2866982,Some positive impact,did not exist,Agree
2866983,Neither positive nor negative impact,did not exist,Neither agree nor disagree


In [7]:
#get rid of the NAs
df.dropna(inplace=True)

<ipython-input-7-e69dbf0ab397>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [8]:
def likert_scale(col):
            
        if col == 'did not exist':
            return 1 
        elif col == 'Strongly disagree':
            return 2
        elif col == 'Disagree':
            return 3
        elif col == 'Neither agree nor disagree':
            return 4
        elif col == 'Agree':
            return 5
        elif col == 'Strongly agree':
            return 6 


def likert_scale2(col):
    
        if col == 'did not exist':
            return 1 
        elif col == 'Extreme negative impact':
            return 2 
        elif col == 'Some negative impact':
            return 3
        elif col == 'Neither positive nor negative impact':
            return 4
        elif col == 'Some positive impact':
            return 5
        elif col == 'Extreme positive impact':
            return 6
      

In [9]:
df['how_much_do_you_agree_with_the_following_statement_u201cvaccines_are_necessary_to_promote_herd_immunity_and_ensure_the_health_and_safety_of_everyone_in_the_community_including_immunocompromised_individuals_u201d'] = df['how_much_do_you_agree_with_the_following_statement_u201cvaccines_are_necessary_to_promote_herd_immunity_and_ensure_the_health_and_safety_of_everyone_in_the_community_including_immunocompromised_individuals_u201d'].apply(lambda x: likert_scale(x))

<ipython-input-9-89602be9b0e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['how_much_do_you_agree_with_the_following_statement_u201cvaccines_are_necessary_to_promote_herd_immunity_and_ensure_the_health_and_safety_of_everyone_in_the_community_including_immunocompromised_individuals_u201d'] = df['how_much_do_you_agree_with_the_following_statement_u201cvaccines_are_necessary_to_promote_herd_immunity_and_ensure_the_health_and_safety_of_everyone_in_the_community_including_immunocompromised_individuals_u201d'].apply(lambda x: likert_scale(x))


In [10]:
df['how_much_do_you_agree_with_the_following_statement_u201ci_am_well_informed_about_covid_19_and_understand_where_to_get_additional_informationu201d'] = df['how_much_do_you_agree_with_the_following_statement_u201ci_am_well_informed_about_covid_19_and_understand_where_to_get_additional_informationu201d'].apply(lambda x: likert_scale(x))

<ipython-input-10-6fd01f088292>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['how_much_do_you_agree_with_the_following_statement_u201ci_am_well_informed_about_covid_19_and_understand_where_to_get_additional_informationu201d'] = df['how_much_do_you_agree_with_the_following_statement_u201ci_am_well_informed_about_covid_19_and_understand_where_to_get_additional_informationu201d'].apply(lambda x: likert_scale(x))


In [11]:
df['how_much_of_an_impact_do_you_believe_the_covid_19_pandemic_and_subsequent_quarantine_measures_have_had_on_your_mental_and_emotional_health'] = df['how_much_of_an_impact_do_you_believe_the_covid_19_pandemic_and_subsequent_quarantine_measures_have_had_on_your_mental_and_emotional_health'].apply(lambda x: likert_scale2(x))

<ipython-input-11-84520b4a3b6f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['how_much_of_an_impact_do_you_believe_the_covid_19_pandemic_and_subsequent_quarantine_measures_have_had_on_your_mental_and_emotional_health'] = df['how_much_of_an_impact_do_you_believe_the_covid_19_pandemic_and_subsequent_quarantine_measures_have_had_on_your_mental_and_emotional_health'].apply(lambda x: likert_scale2(x))


In [12]:
df

,how_much_of_an_impact_do_you_believe_the_covid_19_pandemic_and_subsequent_quarantine_measures_have_had_on_your_mental_and_emotional_health,how_much_do_you_agree_with_the_following_statement_u201cvaccines_are_necessary_to_promote_herd_immunity_and_ensure_the_health_and_safety_of_everyone_in_the_community_including_immunocompromised_individuals_u201d,how_much_do_you_agree_with_the_following_statement_u201ci_am_well_informed_about_covid_19_and_understand_where_to_get_additional_informationu201d
0,4,1,4
1,2,1,5
2,3,1,5
3,3,1,5
4,3,1,5
...,...,...,...
2866980,4,1,4
2866981,3,1,5
2866982,5,1,5
2866983,4,1,4


In [38]:
#whether if it's suitable to conduct factor analysis 
corr=list(df.corr().to_numpy())
bartlett(*corr)

BartlettResult(statistic=0.045756812588736, pvalue=0.9773813199647742)

In [36]:
kmo = calculate_kmo(df.values)
kmo[1]

0.5018613437102944

In [16]:
df

,how_much_of_an_impact_do_you_believe_the_covid_19_pandemic_and_subsequent_quarantine_measures_have_had_on_your_mental_and_emotional_health,how_much_do_you_agree_with_the_following_statement_u201cvaccines_are_necessary_to_promote_herd_immunity_and_ensure_the_health_and_safety_of_everyone_in_the_community_including_immunocompromised_individuals_u201d,how_much_do_you_agree_with_the_following_statement_u201ci_am_well_informed_about_covid_19_and_understand_where_to_get_additional_informationu201d
0,4,1,4
1,2,1,5
2,3,1,5
3,3,1,5
4,3,1,5
...,...,...,...
2866980,4,1,4
2866981,3,1,5
2866982,5,1,5
2866983,4,1,4


In [29]:
std = StandardScaler()
df_zs = std.fit_transform(df)
df_zs

array([[ 0.93136261, -0.21677464, -0.96602883],
       [-0.82011184, -0.21677464,  0.16171502],
       [ 0.05562538, -0.21677464,  0.16171502],
       ...,
       [ 1.80709984, -0.21677464,  0.16171502],
       [ 0.93136261, -0.21677464, -0.96602883],
       [ 0.05562538, -0.21677464,  0.16171502]])

In [32]:
df_zs_cov = np.cov(df_zs.T)

In [39]:
e , ev = nlg.eig(df_zs_cov)

eig = pd.DataFrame()
eig['names'] = df.columns
eig['e'] = e
eig.sort_values('e', ascending=False, inplace=True)
print("\n特征值\n：",eig)
#查看特征向量，一行为一个特征向量
eig1=pd.DataFrame(ev)
eig1.columns = df.columns
eig1.index = df.columns
print("\n特征向量\n",eig1)



特征值
：                                                names         e
0  how_much_of_an_impact_do_you_believe_the_covid...  1.261706
2  how_much_do_you_agree_with_the_following_state...  0.996246
1  how_much_do_you_agree_with_the_following_state...  0.742049

特征向量
                                                     how_much_of_an_impact_do_you_believe_the_covid_19_pandemic_and_subsequent_quarantine_measures_have_had_on_your_mental_and_emotional_health  \
how_much_of_an_impact_do_you_believe_the_covid_...                                           0.703597                                                                                            
how_much_do_you_agree_with_the_following_statem...                                           0.131621                                                                                            
how_much_do_you_agree_with_the_following_statem...                                          -0.698303                                                   

In [42]:
w = list()
for i in range(len(e)):
    w.append(eig['e'][i] / eig['e'].sum())
print(w)
#计算特征值的累计贡献率
q = list()
for j in range(len(e)):
    q.append(eig['e'][:j].sum() / eig['e'].sum())
print(q)
# 求公因子个数m,使用前m个特征值的比重大于80%的标准，选出了公共因子,根据累计贡献率得出
for m in range(len(e)):
    print(eig['e'][:m].sum())
    print(eig['e'].sum())
    print(eig['e'][:m].sum() / eig['e'].sum())
    if eig['e'][:m].sum() / eig['e'].sum() >= 0.8:
        print("\n主成分个数:", m)
        break

[0.42056856261132164, 0.24734961694614088, 0.33208182044253753]
[0.0, 0.42056856261132164, 0.7526503830538591]
0.0
3.0000010464082294
0.0
1.2617061279203698
3.0000010464082294
0.42056856261132164
2.257951936741132
3.0000010464082294
0.7526503830538591


In [51]:
fa = FactorAnalyzer(n_factors=2, rotation='varimax')

In [52]:
fa.fit(df)

FactorAnalyzer(n_factors=2, rotation='varimax', rotation_kwargs={})

In [53]:
pd.DataFrame(fa.loadings_)

,0,1
0,0.493187,0.167443
1,0.021528,0.145415
2,-0.514235,-0.020814


In [54]:
var = fa.get_factor_variance()  
for i in range(0,3):
    print(var[i])

[0.50813459 0.04961581]
[0.1693782 0.0165386]
[0.1693782 0.1859168]


In [55]:
fa.get_communalities()


array([0.27127077, 0.02160891, 0.26487072])

In [57]:
fa_t_score = np.dot(np.mat(df_zs), np.mat(fa.loadings_))
fa_t_score

matrix([[ 0.95143511,  0.14453479],
        [-0.49229491, -0.17221011],
        [-0.0603925 , -0.02557409],
        ...,
        [ 0.80341231,  0.26769796],
        [ 0.95143511,  0.14453479],
        [-0.0603925 , -0.02557409]])

In [58]:
weight = var[1]     
fa_t_score_final = (np.dot(fa_t_score, weight) / e.sum()).real


In [59]:
fa_t_score_final

matrix([[ 0.05451424, -0.02874404, -0.00355071, ...,  0.04683594,
          0.05451424, -0.00355071]])